## R1 To R1 Attractor, Position Around Point

In [27]:
using PBDS, StaticArrays, LinearAlgebra, BenchmarkTools

## Setup

In [28]:
PBDS.metric_chart(xn, task::Attractor{<:PositionAroundPoint{R1,R1}}, CN::Chart{1,R1}) = 
    default_metric(xn, task, CN)
PBDS.potential_chart(xn, task::Attractor{<:PositionAroundPoint{R1,R1}}, CN::Chart{1,R1}) = sum(xn.^2)
PBDS.dissipative_forces_chart(xn, vn, task::Attractor{<:PositionAroundPoint{R1,R1}}, CN::Chart{1,R1}) = -3*vn
PBDS.weight_metric_chart(xn, vn, task::Attractor{<:PositionAroundPoint{R1,R1}}, CN::Chart{1,R1}) = 
    default_weight_metric(xn, vn, task, CN)

In [29]:
xm_goal = SA[0.]
M, N = R1, R1
task = Attractor(PositionAroundPoint{M,N}(xm_goal))
CM, CN = Chart{1,M}(), Chart{1,N}()

(Chart{1, R1}(), Chart{1, R1}())

## Point Acceleration

In [30]:
# Initial state
xm = SA[2.]
vm = SA[2.]

σxddot, = single_task_acceleration(xm, vm, task, CM, CN)

([-10.0], Chart{1, R1}())

## Single Trajectory

In [31]:
Time = 10
dt = 0.05
robot_coord_rep = ChartRep()
traj = propagate_task(xm, vm, task, CM, CN, Time, dt, robot_coord_rep)
traj.xm[end]

1-element SVector{1, Float64} with indices SOneTo(1):
 0.0002863568464694686

In [38]:
using ProgressMeter, VectorizedRoutines
using Plots, Makie, Observables

In [39]:
Nplot = length(traj.xm)
isdefined(Main, :no_plots) || Plots.plot(getindex.(traj.xm,1)[1:Nplot])

## Multiple Trajectories

In [42]:
Time = 5
dt = 0.02
a = 2.
xm, vm = Matlab.meshgrid(-a:1.:a, -a:1.:a)
xm, vm = reshape(xm, length(xm)), reshape(vm, length(vm))

# Remove zero point
ind_mid = length(xm) ÷ 2 + 1 
deleteat!(xm, ind_mid)
deleteat!(vm, ind_mid)

ntraj = length(xm)
trajs = Array{Any}(undef, ntraj)
p = Progress(ntraj)

Threads.@threads for i = 1:ntraj
    trajs[i] = propagate_task(SA[xm[i]], SA[vm[i]], task, CM, CN, Time, dt, robot_coord_rep)
    isdefined(Main, :Test) || next!(p)
end

Progress: 100%|█████████████████████████████████████████| Time: 0:00:00


In [44]:
using GeometryBasics

Makie.inline!(false)
fx(i) = [trajs[j].xm[i][1] for j = 1:ntraj]
fy(i) = [trajs[j].vm[i][1] for j = 1:ntraj]
fz(i) = [trajs[j].am[i][1] for j = 1:ntraj]
f(i) = [trajs[j].xm[i] trajs[j].vm[i] trajs[j].am[i]]
iobs = Observable(1)

ax_size, plot_size = 5, 800
limits = FRect3D((-ax_size, -ax_size, -ax_size), (2*ax_size, 2*ax_size, 2*ax_size))
scene = Scene(resolution = (plot_size, plot_size))
Makie.scatter!(scene, lift(i -> fx(i), iobs), lift(i -> fy(i), iobs), lift(i -> fz(i), iobs))
Makie.scatter!(scene, lift(i -> fx(i), iobs), lift(i -> fy(i), iobs), lift(i -> fz(i), iobs), markersize = ax_size/20, color = :blue, limits = limits)
Makie.scatter!(scene, [xm_goal[1] 0. 0.], markersize = ax_size/20, color = :darkgreen)
for i = 1:ntraj
    Makie.lines!(scene, getindex.(trajs[i].xm,1), getindex.(trajs[i].vm,1), getindex.(trajs[i].am,1), color = :purple, alpha = 0.1, transparency = true)
end

Makie.xlabel!(scene, "x")
Makie.ylabel!(scene, "v")
Makie.zlabel!(scene, "a")
axis = scene[Axis]
axis.showaxis = false
isdefined(Main, :no_plots) || display(scene)

UndefVarError: UndefVarError: `FRect3D` not defined

In [23]:
function record_scene(scene, filename, iobs, N)
    p = Progress(N)
    record(scene, filename, 1:N) do i
        iobs[] = i
        isdefined(Main, :Test) || next!(p)
    end
    isdefined(Main, :Test) || display("text/html", html_video(filename))
end

filename = "R1ToR1Attractor_xva.mp4"
isdefined(Main, :no_plots) || record_scene(scene, filename, iobs, length(trajs[1].xm))

UndefVarError: UndefVarError: `scene` not defined

### In (x, v) space

In [24]:
AbstractPlotting.inline!(false)
fx(i) = [trajs[j].xm[i][1] for j = 1:ntraj]
fy(i) = [trajs[j].vm[i][1] for j = 1:ntraj]
iobs = Observable(1)

ax_size, plot_size = 3, 500
limits = FRect((-ax_size, -ax_size), (2*ax_size, 2*ax_size))
scene = Scene(resolution = (plot_size, plot_size))
Makie.scatter!(scene, lift(i -> fx(i), iobs), lift(i -> fy(i), iobs), markersize = ax_size/20, color = :blue, limits = limits)
Makie.scatter!(scene, [0. 0.], markersize = ax_size/20, color = :darkgreen)
for i = 1:ntraj
    Makie.lines!(scene, getindex.(trajs[i].xm,1), getindex.(trajs[i].vm,1), color = :purple, alpha = 0.1, transparency = true)
end

Makie.xlabel!(scene, "x")
Makie.ylabel!(scene, "v")
axis = scene[Axis]
axis.showaxis = false
isdefined(Main, :no_plots) || display(scene)

UndefVarError: UndefVarError: `AbstractPlotting` not defined

In [25]:
filename = "R1ToR1Attractor_xv.mp4"
isdefined(Main, :no_plots) || record_scene(scene, filename, iobs, length(trajs[1].xm))

UndefVarError: UndefVarError: `scene` not defined